# Tools

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-5")

In [3]:
from langchain_core.tools import tool
    
@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    #query = _safe_sql(query)
    q = query
    if q.startswith("Error:"):
        return q
    try:
        return db.run(q)
    except Exception as e:
        return f"Error: {e}"


In [4]:
from langchain_core.tools import tool
import requests

FALLBACK_RATES = {"USD": 1.0, "EUR": 0.92, "JPY": 150.0, "GBP": 0.79}

@tool
def convert_currency(amount: float, to_currency: str) -> float:
    """Convert an amount in USD to another currency using live exchange rates.
    Always use this tool if the user requests a currency different 
    from the one stored in the database (USD)."""
    try:
        url = f"https://api.exchangerate.host/convert?from=USD&to={to_currency}&amount={amount}"
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        result = data.get("result")
        if result is not None:
            return round(float(result), 2)
    except Exception as e:
        print(f"[convert_currency] Falling back due to error: {e}")

    if to_currency not in FALLBACK_RATES:
        raise ValueError("Unsupported currency in fallback mode")
    return round(amount * FALLBACK_RATES[to_currency], 2)

In [ ]:
SYSTEM = f"""You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[execute_sql, convert_currency],
    prompt=SystemMessage(content=SYSTEM),
)

In [24]:
question = "What is the most costly purchase by Frank Harris in EUR?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What is the most costly purchase by Frank Harris in EUR?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_U9Kgteg1THbuNAuD2nJVQngq)
 Call ID: call_U9Kgteg1THbuNAuD2nJVQngq
  Args:
    query: SELECT name FROM sqlite_master WHERE type='table' AND (
  name LIKE '%customer%' OR name LIKE '%user%' OR name LIKE '%client%' OR
  name LIKE '%order%' OR name LIKE '%purchase%' OR name LIKE '%transaction%' OR name LIKE '%sale%'
) ORDER BY name LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Customer',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_Dvs6ehDhNWaIvhW1TaQttbMI)
 Call ID: call_Dvs6ehDhNWaIvhW1TaQttbMI
  Args:
    query: PRAGMA table_info('Customer');
================================= Tool Message =================

In [ ]:
Lets take a look at our message history


In [26]:
len(steps)

12

In [44]:
def print_message(msg):
    # Get all fields in the Pydantic object as a dict
    data = msg.dict()
    # Drop the noisy ones
    data.pop("response_metadata", None)
    data.pop("usage_metadata", None)
    # Print the rest
    print(f"{msg.__class__.__name__}(")
    for k, v in data.items():
        print(f"  {k}={v!r},", end ="")
    print(")")



In [45]:
for m in steps[-1]["messages"]:
    print_message(m)

HumanMessage(
  content='What is the most costly purchase by Frank Harris in EUR?',  additional_kwargs={},  type='human',  name=None,  id='70bff948-071f-4ad2-a275-faa4a32e065c',)
AIMessage(
  content='',  additional_kwargs={'refusal': None},  type='ai',  name=None,  id='lc_run--16a888e6-3431-4d58-baaf-2d0c3a308590-0',  tool_calls=[{'name': 'execute_sql', 'args': {'query': "SELECT name FROM sqlite_master WHERE type='table' AND (\n  name LIKE '%customer%' OR name LIKE '%user%' OR name LIKE '%client%' OR\n  name LIKE '%order%' OR name LIKE '%purchase%' OR name LIKE '%transaction%' OR name LIKE '%sale%'\n) ORDER BY name LIMIT 5;"}, 'id': 'call_U9Kgteg1THbuNAuD2nJVQngq', 'type': 'tool_call'}],  invalid_tool_calls=[],)
ToolMessage(
  content="[('Customer',)]",  additional_kwargs={},  type='tool',  name='execute_sql',  id='3a3f4b51-ae8c-45ce-bb0d-471cb8cf1b18',  tool_call_id='call_U9Kgteg1THbuNAuD2nJVQngq',  artifact=None,  status='success',)
AIMessage(
  content='',  additional_kwargs={'refu

/var/folders/jz/2tg__xkj7fq6_dzmcc85dlvh0000gn/T/ipykernel_81711/400630682.py:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  data = msg.dict()


In [43]:
for i, step in enumerate(steps):
    print(f"========== step{i} ===============")
    for m in step["messages"]:
        print_message(m0


_IncompleteInputError: incomplete input (3488395516.py, line 4)